# Momsitter User Sequence Analysis

이 노트에서는 사용자의 네비게이팅 정보, 액션 로그, API usage, Web log 등의 아이템을 직렬화된 시퀸스 데이터로 만든 다음, 데이터 마이닝을 통한 분석을 시도한다. 첫번째 섹션에서는 가장 베이직한 아이디어를 바탕으로 의미있는 결과가 추출가능한가(기본 범주와 타겟 범주의 구분 가능성)를 목표로 한다. 시퀸스 마이닝에서는 SPADE 나 WTP 등 많은 알고리즘이 있지만, 본 노트에서는 간략하게 데이터를 탐험해보기 위해 ngram 을 이용해서 분석을 해본다.

목표: 보편적으로 많이 나타나는 ngram 추출

키워드: SPADE, WTP, Sequence mining, Web usage mining

## Data munging

이 노트에서 사용할 데이터는 이미 가공이되어 있는 상태이다. 데이터는 \[SID, Sequence\] 로 이루어진 csv 데이터를 사용하고 있다. 추가적으로 데이터를 가공하는 작업 없이 바로 사용할 수 있다고 가정한다.

### Load the data

미리 가공되어 있던 csv 데이터를 로드한다. 해당 csv 에는 SID, Sequence 정보가 담겨있다.

In [4]:
import csv
import pandas as pd

seq_data = pd.read_csv('./data/IP_GROUP_BY_SEQ.csv')
seq_data.head(10)

,ID,IP_SPLIT_RESULT,SEQUENCE_PARSE
0,1,119.67.115.197,"<87,89,80,26,1,132,93,132,93,63,1,90,92>"
1,2,119.67.115.197,"<23,23,90>"
2,3,211.104.55.131,"<1,26,26,1,93,93,1,5,1,93,132,93,132,93,93,93,..."
3,4,1.235.19.6,"<1,90,92,1,132,93,93,93,14,93,132,93,132,93,13..."
4,5,1.235.19.6,"<87,89,80,29,132,113,132,113,132,92,93,93,93,1..."
5,6,66.249.69.29,<1>
6,7,46.229.168.78,<1>
7,8,46.229.168.71,<1>
8,9,46.229.168.72,<1>
9,10,46.229.168.66,<1>


### Define util analysis function

분석에 사용하기 위한 유틸 함수를 선언한다. 

In [5]:
# 시퀸스의 시작, 종료 토큰을 제외한 아이템을 콤마(',') 를 기준으로 분리하여
# 아이템 배열을 만들어서 리턴해준다
def sequence_tokenize(seq):
    return seq.replace('<', '').replace('>', '').split(',')

In [12]:
# bi, tri 등 어떤 ngram 을 만들지와 시퀸스 하나를 받아
# ngram dictionary 를 만들어서 리턴해준다
def extract_ngram(n, seq_items):
    result = []
    for idx, item in enumerate(seq_items):
        window_set = seq_items[idx:idx + n]
        if len(window_set) == n:
            result.append(','.join([str(x) for x in window_set]))
    return result

In [10]:
from collections import defaultdict

# 전에 data set 을 받아 ngram 을 만들어 리턴
def extract_ngram_set(n, sdb):
    # 아직 나타나지 않은 ngram 을 0 값에서 시작시키기 위해 디폴트 값을 사용
    result = defaultdict(int)
    for index, row in sdb.iterrows():
        # ngram 을 추출
        ngram = extract_ngram(n, sequence_tokenize(row['SEQUENCE_PARSE']))
        # ngram dictionary 에 저장
        for item in ngram:
            result[item] += 1
    return pd.DataFrame(result.items(), columns=['sid', 'support'])

## Data mining

구현한 유틸 함수를 바탕으로 bi-gram 을 먼저 추출해보고 더 큰 n 값을 바탕으로 ngram 을 추출해보도록 한다.

### Make bi-gram

In [13]:
bi_gram = extract_ngram_set(2, seq_data)

In [15]:
# 가장 많이 나타나는 패턴 순으로 확인하기 위해 support 값 순으로 정렬
bi_gram.sort_values(['support'], ascending=[False]).head(10)

,sid,support
10,"1,5",234607
1415,"5,1",233813
1524,"31,113",196161
512,"113,31",166227
354,"113,113",162524
846,"93,93",161214
462,"132,93",125325
1665,"93,132",111810
682,"113,132",102770
693,"132,132",102234


### Make ngram

In [16]:
tri_gram = extract_ngram_set(3, seq_data)
four_gram = extract_ngram_set(4, seq_data)
five_gram = extract_ngram_set(5, seq_data)

In [17]:
tri_gram.sort_values(['support'], ascending=[False]).head(10)

,sid,support
33975,"1,5,1,5,1",225945
31721,"5,1,5,1,5",224531
71051,"113,31,113,31,113",73861
34886,"93,93,93,93,93",45486
76985,"31,113,31,113,31",45028
29021,"93,32,93,32,93",36371
21799,"113,113,113,113,113",29137
31982,"31,113,113,31,113",28156
18149,"113,31,113,113,31",25919
87169,"31,113,31,113,113",25706


In [18]:
four_gram.sort_values(['support'], ascending=[False]).head(10)

,sid,support
5647,"1,5,1,5",228166
8918,"5,1,5,1",226567
4046,"31,113,31,113",87237
4810,"113,31,113,31",75831
20237,"93,93,93,93",63689
36519,"113,113,31,113",54019
5578,"113,31,113,113",49620
19772,"113,113,113,113",44757
15990,"32,93,32,93",41838
11446,"132,93,132,93",41317


In [20]:
five_gram.sort_values(['support'], ascending=[False]).head(20)

,sid,support
33975,"1,5,1,5,1",225945
31721,"5,1,5,1,5",224531
71051,"113,31,113,31,113",73861
34886,"93,93,93,93,93",45486
76985,"31,113,31,113,31",45028
29021,"93,32,93,32,93",36371
21799,"113,113,113,113,113",29137
31982,"31,113,113,31,113",28156
18149,"113,31,113,113,31",25919
87169,"31,113,31,113,113",25706


## Analysis & Conclustion

대부분의 사용자가 다른 사용자를 검색하고 상세 정보를 조회하는 시퀸스에 초점이 맞추어져 있다. 또한 몇회의 상세 정보 조회 후 신청 & 조회를 하는 패턴을 보인다. 하지만 우리가 추출해낸 시퀸스 만으로는 더 의미있는 정보를 알기는 힘들다. 더 양질의 솔루션을 얻기 위해서는 다음 태스크가 선행되어야 할 것으로 보인다.

* 반복되는 시퀸스의 그룹핑
* 의미없는 시퀸스 필터링
* 유사 시퀸스의 머징